In [1]:
import pandas as pd
import datetime
import subprocess
import os
from pathlib import Path

# Process *les journaux officiels*

In [4]:
tmpfile = "tmp.txt"
df_file = "data_processed.csv"
if os.path.isfile(df_file):
    df = pd.read_csv(df_file)
    jo_list = sorted(df["jo"].unique())
else:
    df = pd.DataFrame(columns=["numero", "série", "sexe", "age", "dep", "jo", "delai"])
    jo_list = []

for decret in Path(".").glob("decrets/*.pdf"):
    # Skip if current JO is already treated
    jo = str(decret).partition("joe_")[2].partition("_")[0]
    if int(jo) in jo_list:
        continue
    
    print(f"Process JO {jo}...")
    date_jo = datetime.datetime.strptime(jo, "%Y%m%d")
    with open(tmpfile, "w") as f:
        subprocess.check_call(["pdf2txt.py", decret], stdout=f)
#         subprocess.check_call(["python", "D:/Programs/Miniconda3/Scripts/pdf2txt.py", str(decret)], stdout=f)
    
    with open(tmpfile) as f:
        content = f.read()
        keys = ["\n\n", ", \n"]
        for key in keys:
            content = content.replace(key, "\n")
        content = content.replace("\ndép", ", dép")
        content = content.replace("dép. \n", ", dép. ")
        content = content.replace("X \n", "X")
        content = content.replace(". \n", "^")
        content = content.replace(": \n", "^")
        content = content.replace("\n", "")
        content = content.replace("^", ".\n")
        lines = content.split("\n")
    
    for line in lines:
        if "NAT," not in line:
            continue
        else:
            if "née" in line:
                sexe = "F"
            else:
                sexe = "H"
            line = line.replace(" au ", " à ").replace(" aux ", " à ")
            naissance = line.partition(" le ")[2].partition(" à ")[0].replace(" ", "")
            try:
                date_naissance = datetime.datetime.strptime(naissance, "%d/%m/%Y")
            except ValueError:
                continue
            age = (date_jo - date_naissance).days / 365.2425
            dep = line.partition("dép.")[2].partition(",")[0].replace(" ", "")
            if len(dep) == 0:
                continue
            elif "Dt" in dep:
                dep = dep.partition("Dt")[0]
            numero = line.partition("NAT,")[2].partition(",")[0].replace(" ", "")
            serie = numero[:8]
            year = int(serie.partition("X")[0])
            weeknum = int(serie.partition("X")[2][1:3]) + 1
            if weeknum > 53:
                continue
            else:
                date_serie = datetime.datetime.strptime(f"{year} {weeknum}-3", "%G %V-%w")
                delai = (date_jo - date_serie).days / (365.2425 / 12)
            line = {"numero": numero, "serie": serie, "sexe": sexe, "age": age, "dep": dep, "jo": jo, "delai": delai}
            df = df.append(line, ignore_index=True)

if os.path.isfile(tmpfile):
    os.remove(tmpfile)

Process JO 20180504...
Process JO 20180901...
Process JO 20180909...
Process JO 20180516...
Process JO 20180209...
Process JO 20180202...
Process JO 20180304...
Process JO 20180427...
Process JO 20180316...
Process JO 20181205...
Process JO 20180829...
Process JO 20180912...
Process JO 20180930...
Process JO 20180328...
Process JO 20180708...
Process JO 20181024...
Process JO 20181228...
Process JO 20180803...
Process JO 20180520...
Process JO 20181222...
Process JO 20181118...
Process JO 20180630...
Process JO 20180615...
Process JO 20180714...
Process JO 20180225...
Process JO 20180824...
Process JO 20180728...
Process JO 20180118...
Process JO 20180309...
Process JO 20180602...
Process JO 20180609...
Process JO 20180927...
Process JO 20180627...
Process JO 20180408...
Process JO 20181110...
Process JO 20180126...
Process JO 20181208...
Process JO 20180216...
Process JO 20180422...
Process JO 20180920...
Process JO 20181223...
Process JO 20181014...
Process JO 20180330...
Process JO 

In [5]:
df.tail()

,numero,serie,sexe,age,dep,jo,delai
63078,2018X012241,2018X012,H,33.495554,075,20180720,3.745457
63079,2017X109560,2017X109,F,48.318583,093,20180720,16.394587
63080,2018X006248,2018X006,H,22.448100,078,20180720,5.125362
63081,2018X014893,2018X014,H,27.422877,095,20180720,3.285488
63082,2017X024224,2017X024,F,51.029111,034,20180720,12.944824


In [6]:
df.isnull().values.any()

False

In [7]:
df.to_csv("data_processed.csv", index=False)